# Extraction des données

Dans le vaste domaine de la data ou des données, il y a toujours une phase qui est indispensable : c'est avoir les données. Or, pour avoir les données, il faut les avoir quelque part. Soit recueillir directement auprès des utilisateurs, consommateurs, soit à partir d'une base de données que vous aviez déjà mise en place, soit en seconde main... 
Notre objectif est d'extraire des données sur au moins un site web. Cette extraction concerne des produits sur différents sites e-commerce pour un but pédagogique et non commercial.

Dans ce projet, nous nous proposons d'étudier les produits des site e-commerces. Nous voulons étudier entre autre quel produit a le prix le plus avantageux pour le consommateur suivant les sites, les tendances des produits, le prix d'un produit selon qu'il soit en promotion ou non. je fais l'extraction des données. 

Pour ce faire, nous avons besoin de passer la première étape de l'analyse de données qui est avoir les données. Or, pour avoir les données, il faut les avoir quelque part. Soit les collecter soi-même, soit elles sont dans des bases de données, dans des datawarehous, dans ce cas on peut faire un dump (à expliquer), soit les récupérer sur des sites dans fichiers csv. Tout cela a un nom qui est l'extraction de données.

L'extraction des données est un processus qui permet de collecter des données sur divers sources dans un but de les stocker ou des les traiter pour plus tard. Les données peuvent être publiques ou privées. Ainsi, les données peuevnt être soumis à des licences publiques ou privées. Les sources de données peuvent très nombreuses, comme des site gouvernementaux, sur des site comme google, des plateformes comme kaggle, avec des licences publiques...

Il y a différents techniques de collecter ou d'extraire des données, comme le web scraping, qui diffèrent du simple récupération d'un fichier csv sur un site. Dans ce projet, c'est justement le web scraping que nous allons utiliser pour récupérer des données sur des sites e-commerces. 

Nous voulons faire ce qui est communément appelé par le web scraping.


## Web Scraping et éthique

Le web-scraping est une technique d'extraction de données, un traitement automatisé qui consiste à collecter des données sur un site web. Alors, il faut préciser d'emblée que tous les site web n'autorisent pas la collecte de leurs données. D'où l'importance de vérifier la politique du site web avant de procéder à la collecte de ses données pour savoir s'il autorise ou non le web scraping. 
D'abord, j'ai souhaité inclure Amazon, car étant l'un des site de vente les plus populaire, et donc pouvant donner la possibilité d'avoir beaucoup données, dans les sites sur lequels je collecte des données, mais après des recherches si il autorise ou pas le web scraping, je me suis rendu compte que partiquement pour l'écrasante majorité des attributs que nous avons besoin, il n'autorise pas les web scraping.  On peut voir ces chose en consultant le robot.txt du site et les conditions générales d'utilisation (CGU). 

### Analyse des fichiers du site
#### le fichier robots
Si vous voulez, ce fichier est comme un panneau de signalisation indiquant les chemins à ne pas prendre, parfois ce panneau concerne toute voiture, parfois est spécifique à certaines voiture, par exemple les poids lourds.
Par exemple, pour Amazon, j'ai consulté https://www.amazon.com/robots.txt. S'il y a des parties que vous ne comprenez pas. C'est tout à fait normal, vous pouvez vous avec chatGPT. J'ai également regardé leurs conditions générales et d'utilisation, en cherchant avec les mot clés "robot" ou "collecte", il ne l'autorise également pas sans un accord express et écrit. J'ai également utilisé chatGPT pour voir si ses résultats correspondent à certains de mes résultats.

Finalement, j'ai me suis dirigé vers d'autres site qui permettent cela et j'ai inclus aussi pour avoir plus des données des API. Ainsi, 

#### Les conditions générales d'utilisation (CGU)
Pour les CGU, j'ai commencé à lire les CGU des sites qui peuvent m'intéresser dans le cadre de ce projet, notamment les parties susceptibles de contenir les passages que je cherche, en particulier ceux sur le web scraping. J'utilise parfois chatGPT pour qu'il interprète certains passages ou qu'il aille chercher les CGU du site, sur lequel je souhaite approfondir mes connaissances sur ces CGU, et de porter une attention particulière à certains mots clés. Il a fait de belles choses. Mais, comme généralement, avec les IA génératives, il ne faut pas s'empêcher d'aller vérifier certaines de leurs informations. Ainsi, s'il me donne son résumé sur ces passages, je peux aller regarder les CGU du site concerné, une fois que je sais que ce passage y existerait, vérifiant en même temps ce qui y est dit exactement. C'est de cette manière que j'ai procédé avec les CGU du site temu.Toujours, dans le cadre de l'extraction des données sur ce projet, en particulier dans celui du web scraping, j'ai voulu faciliter la tâche en faisant un programme qui analyse les CGU du site sur lequel je souhaite faire du scraping en cherchant des mots clés et des passages qui autorisent ou non le web scraping, comme celui sur le fichier robots du site. Contrairement au programme sur le fichier robots.txt où l'utilisateur ne donne que le domaine du site et une liste de mots clés, dans celui pour les CGU, il doit fournir le domaine du site et le chemin relative des CGU et une liste de mots clés. Je l'ai fait ainsi parce qu'on n'a pas les mêmes chemins relatifs des CGU selon les sites, ni les mêmes nomination. Parfois, c'est/CGU, parfois /policies, sur d'autres sites c'est /help/... Les mots clés sont ce que j'ai jugé au travers des lectures de CGU mais j'ai également utilisé chatGPT pour les mots clés qui sont plus pertinents à rechercher dans les CGU concernant le web scraping. Enfin, j'ai fais un tri pour choisir les plus pertinents de ceux issus des lectures de CGU et ceux proposés par chatGPT.Donc, si l'utilisateur fournit ces deux chemins et la liste des mots à cherché, le programme analyse le contenue identifie la présence de ses mots ainsi que leurs passage. S'il les trouve, il les affiche. Ce programme permet de faciliter la recherche de certains mots clés, des passages qui doivent attirer l'attention de celui qui souhaite scraper un site. Cependant, la non exhaustivité des termes de la liste peut être une limite pour ce programme.

In [1]:
# import libraries
import requests
import mechanicalsoup
from bs4 import BeautifulSoup
from urllib.request import urlopen
#from playwright.sync_api import sync_playwright
from playwright.async_api import async_playwright, TimeoutError
from urllib.parse import parse_qs, urlparse
import asyncio
#import re
from rich import print
import json
import csv
from fake_useragent import UserAgent

In [2]:
# Préalables pour le web scraping
# analyse du fichier robots
def robots_file_analysis(base_url):
    """
        Le fichier robots.txt contient des chemins que les bots ou robots
        ne doivent pas utiliser. Ce paneau d'attention est parfois addressé
        à tout robot, parfois à des robots spécifiques pour leur dire de ne pas
        emprunter ces chemin. Ainsi, ici, j'analyse les chemins que le site ne veulent
        pas mon programme emprunte pour y extraire des données. Ces chemins contiennent des 
        mots clés qui nous intéressent dans notre scraping. S'ils y sont présent, et que c'est 
        qu'ils sont précédés par un "Disallow", on pourrait ne pas estraire ces données. En revanche, 
        s'ils sont précédés par un "Allow", alors on pourrait les extraire de ce site.
        Par exemple, dans notre présent projet, nous voulons extraire du site les produits et certains de 
        leur caractérisques. ALors "produit" est un mot clé pour nous. Si un chemin contient ce mot clé dans 
        le fichier robots.txt, alors notre programme le détecte et l'affiche. Une fois qu'on a ce chemin, on peut
        voir s'il s'agit de tous les produits ou certains de ces caractéristiques.
        
        Cette fonction --robots_file_analysis-- prend l'URL de base du site qu'on souhaite extraire ces données, on y ajoute 
        le chemin vers le fichier "/robots.txt" qu'on souhaite analyser, et renvoie un dictionnaire
        contenant les user-agent, comme clés, et les chemins comme valeurs.
 
    """
    
    headers = {"user-agnet":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:143.0) Gecko/20100101 Firefox/143.0"}

    #response = requests.get("https://httpbin.io/user-agent")
    # Envoi d'une requête au site
    response = requests.get(base_url+"/robots.txt", headers=headers)

    # On construit une liste des lignes du fichier robots
    robots_lines_list = response.text.lower().splitlines()
    #print(robots_text_lines)
    
    # On récupère tous les user-agent du robots.txt
    userAgent_list = []
    for i, agent in enumerate(robots_lines_list):
        userAgent_indice = agent.find("user-agent")
        if userAgent_indice != -1:
            userAgent_list.append(agent)
    # print(userAgent_list)
    
    # On construit un dictionnaire avec comme key le user-agent et value
    # l'item qui ne doit être aspiré
    robots_text_dict = {}
    # On construit un dictionnaire avec comme key le user-agent et value
    # l'item qui ne doit être aspiré
    for i, userAgent in enumerate(userAgent_list):
        start = robots_lines_list.index(userAgent)+1
        
        if i+1 < len(userAgent_list):
            end = robots_lines_list.index(userAgent_list[i+1])
            robots_text_dict[userAgent] = robots_lines_list[start:end]
        else:
            robots_text_dict[userAgent] = robots_lines_list[start:]
    
    return robots_text_dict


def robots_results(base_url, robots_keywords_list):
    """
        Cette fonction -- robots_results -- appelle la --robots_file_analysis--. Elle aussi prend 
        l'URL de base du site, avec le chemin robots.txt, et une liste de mots clés. Elle renvoie 
        les résultats dans un dictionnaire contenant les user-agent, comme clés, concatenés avec le mot clé recherché, 
        séparés par un "_", et comme valeurs les chemins trouvés dans lesquels il y a le mot clé qu'on
        recherche.       
        
        J'ai paramétré le mots clés, car nous ne chercons pas forcément à extraire les mêmes choses. 
        Ainsi, l'utilisateur peut mettre les mots sur lesquels il souhaite travailler. Pour moi, par exemple,
        parmi les mots qui m'interresse figurent "produit", "product", "catalogue", ...
    """
    results_dic = robots_file_analysis(base_url)
    
    results = {}
    for i, item in enumerate(results_dic.items()):
        # On affiche les keywords présent dans les values de la clé
        for robots_keyword in robots_keywords_list:
            for j, robots_keyword_path in enumerate(item[1]):
                if robots_keyword in robots_keyword_path:
                    results.setdefault((item[0]+"_"+robots_keyword).replace(" ", ""), []).append(robots_keyword_path)
                else:
                    continue
                    
    return results
    

def display_robots_results(base_url, robots_keywords_list):
    
    """
        Cette méthode -- display_robots_results -- appelle la fonction -- robots_results -- 
        et permet d'afficher les résultats obtenus. Si aucun des mots clés n'est trouvé, elle
        nous affiche le message "Aucun de ces mots clés que vous cherchez n'est trouvé dans le fichier /robots.txt !"
    """
    
    results_dic = robots_results(base_url, robots_keywords_list)
    
    #print(results_dic)
    #print(results_dic.keys())
    if results_dic == {}:
        print("Aucun de ces mots clés que vous cherchez n'est trouvé dans le fichier /robots.txt !")
    else :
        userAgent_list = []
        for key, values in results_dic.items():
            userAgent_key = key.split("_") # le key avec split à partir "_" -> liste de deux mots : user-agent et le mot clé qui est cherché 
            if userAgent_key[0] not in userAgent_list:
                userAgent_list.append(userAgent_key[0])
        print(userAgent_list)        
           
        for key, values in results_dic.items():
            userAgent_key = key.split("_")
            
            if userAgent_key[0] in userAgent_list:
                print(f"{userAgent_key[0]}")
                userAgent_list.remove(userAgent_key[0])
                
            print(f"\t {userAgent_key[1]} est bien présent dans : ")
            
            for i, robots_keyword_path in enumerate(values):
                print("\t\t", robots_keyword_path)
                
    # lien vers robots.txt et les CGU
    print("Pour plus d'informations, vous pouvez consulter : ", base_url+"/robots.txt")

In [3]:
# Analyse du fichier conditions générales d'utilisation (CGU)
def cgu_analysis(base_url, cgu_path):
    headers = {"user-agnet":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:143.0) Gecko/20100101 Firefox/143.0"}

    # France
    proxies = {
    "http": "https://89.168.33.135:3217",
    "https": "https://89.168.33.135:3217"
    }
    
    
    # keywords = "cgu", "cgv", "conditions", "utilisation", "vente", "terms", "terms"
    #                "service", "privacy", "policy", "legal", "mention"
    print(base_url+cgu_path)
    response = requests.get(base_url+cgu_path, headers=headers) # proxies=proxies
    print(response.status_code)
    #print(response.history)
    #print(response.text)
    #session = requests.Session()

    #page = urlopen(base_url)
    #html_text = page.read().decode("utf-8")
    #print(html_text) # avec amazon, on a cette erreur : 
    # TypeError: urlopen() got an unexpected keyword argument 'headers'

    soup = BeautifulSoup(response.text, "html.parser")
    print(soup.get_text())


    # "propiété", "propriété intellectuelle", "licence", "licence d'accès", 
    # "licence d'usage", "accès aux données", "license", "data access", "usage personnel",
    # "usage non personnel", "usage commercial", "usage non commercial", "partie substantielle",
    # "partie non susbtantielle"

In [4]:
# fonction pour supprimer les balises html
def remove_tags(soup_find_all_list):
    texts_list = []
    for text in soup_find_all_list:
        text = text.get_text(strip=True)
        texts_list.append(text)

    return texts_list


async def cgu_analysis_auto(base_url, cgu_path, keywords_list):
    """
        Cette fonction prend la base de l'url d'un site web, le chemin relatif
        de l'url des conditions générales d'utilisation et une list de de mots clés,
        pour vérifier si ces dernisers sont présents ou pas dans le contenu des cgu.
        Elle récupère le contenu d'une page html, notamment les cgu, et retourne 
        un dictionnaire avec le mot clé et le paragraphe dans lequel il se trouve.
    """
    async with async_playwright() as pw:
        browser = await pw.chromium.launch(headless=False)
        #browser = await pw.firefox.launch(headless=False)
        page = await browser.new_page()
        
        async def handle_response(response):
            url = response.url
            try:                
                if "api" in url or url.endswith("json"):
                    data = await response.json
                    print("API trouvé : ", url)
                    print(data)
            except:
                pass
                #print("Pas d'API trouvé !")

        # La page écoute l'évenenment "response", si le navigateur reçoit des réponses, 
        # elle appelle un callback en l'occurrence handle_response qui traite les responses reçues et 
        # les trie selon qu'ils soient .json ou non
        #page.on("response", handle_response)
        page.on("response", f=lambda response: handle_response(response))

        
        # La réponse de la navigation
        response = await page.goto(base_url+cgu_path, wait_until="networkidle")
        #response = await page.goto(base_url+cgu_path, {timeout:0}, wait_until="networkidle")
        page.set_default_navigation_timeout(0)
        status = response.status if response else None
        print("Status HTTP : ", status)
        
        if status != 200:
            print("Une erreur est survenue ! \nVeuillez réssayer plus tard" +
            "\nou accédez au lien ", base_url+cgu_path)

        #print(response.url)
        #print(response.json)
        await page.wait_for_selector("body")
        content = await page.content()
        #print(content) # [:2000]

        # On récupère le texte des cgu avec BeautifulSoup
        soup = BeautifulSoup(content, "html.parser")
        #print(soup.get_text())
        await asyncio.sleep(5)
        #await browser.close()
        
        
        # vérifier si un keyword est présent dans les titre ou sous-titres
        # vérifier si un keyword est présent dans les paragraphes
        h2_titles_list = soup.find_all("h2")
        #print(h2_titles_list)
        h3_titles_list = soup.find_all("h3")
        p_list = soup.find_all("p")
        li_list = soup.find_all("li")

        # tags deletion
        h2titles_list = remove_tags(h2_titles_list)
        h3titles_list = remove_tags(h3_titles_list)
        paragraphs_list = remove_tags(p_list)
        puces_list = remove_tags(li_list)
        

        keywords_dic = {}
        for keyword in keywords_list:
            for paragraph in paragraphs_list:
                if keyword in paragraph:                    
                    keywords_dic.setdefault(keyword, []).append(paragraph)
            for puce in puces_list:
                if keyword in puce:
                    keywords_dic.setdefault(keyword, []).append(puce)

        return keywords_dic


async def display_cgu_results(base_url, cgu_path, keywords_list):
    """
        Affichage des mots clés trouvés dans les cgu de l'url et 
        leurs paragraphes
    """
    keywords_dic = await cgu_analysis_auto(base_url, cgu_path, keywords_list)

    # On met les mots clés qui sont présent dans le passage des cgu
            # en gras et de couleur rouge 
    values_list = [] #list(keywords_dic.values())
    for key, values in keywords_dic.items():
        #print(type(values), "-- ", values)
        #print("***","[bold]"+key+"[/bold]", " ---> ", values)
        print("***","[bold]"+key+"[/bold]", " ---> ")

        for value in values:
            value = value.replace(key, "[bold red]"+key+"[/bold red]")
            print("\t", value)
  
    #     print()

    print("Pour plus d'informations, vous pouvez consulter : ", base_url+cgu_path)

    # print("Pour plus d'informations, vous pouvez consulter : ", base_url+cgu_path) 


# Si vous mettez un nombre important de à chercher, vous aurez beaucoup de passages. 
# Egalement, si vous mettez un mot clé seul qui est abondamment cité dans les cgu, 
# comme licence ou utilisation, vous aurez beaucoup de passages avec ces mots, qui peuvent
# ne pas être pertinents pour votre recherche. Donc, l'idéal c'est de combiner les mots clés
# importants. Par exemple, on peut associer "utilisation" et ..., extraction et données en 
# "extraction de données" pour avoir des passages pertients.
# En revance, certains mots clés sont à eux seuls pertients dans la recherche, comme "scraping", 
# "scraper", "robot" (bon, on suppose qu'on ne va pas parler de robots de cusine dans les cgu),
# "bot", ...

# Enfin, bien que ce programme facilite la recherche d'un mot clés dans les cgu et leur lecture, 
# il est toujours intéressant de se rapporter sur le lien des cgu que nous affichons dans les réponse
# pour de plus amples informations, peut-être sur les passage qui sont affichés. Si vous ne trouvez pas un mot, cela ne veut pas forcément dire qu'il n'exite pas dans les cgu.
# peut-être les passages des cgu contenant le mot que vous cherché est inclus dans d'autres balises 
# différents de ceux qui sont habituellement utilisés pour un texte. D'où l'importance de jetter un coup
# d'oeil sur l'url des cgu que nous affichons pour plus d'informations.

In [5]:
# Pour s'assurer que toutes les pages de l'url sont chargées
# 
async def scroll_to_bottom(page, n):
    last_height = await page.evaluate('document.body.scrollHeight')
    i = 1

    while i <= n:
        print(f"Scrolling page ({i})...")
        # On scrolle jusqu'au bas de la page
        await page.evaluate('window.scroll(0, document.body.scrollHeight);')
        # On demande au programme d'attendre 1s pour que la page puisse se charger
        await asyncio.sleep(1)

        # On calcul le nouveau scroll (le scroll actuel) et le compare à l'ancien
        new_height = await page.evaluate('document.body.scrollHeight')
        # On les compare s'ils sont les mêmes alors on sort du boucle
        if new_height == last_height:
            break

        last_neight = new_height
        i += 1
        

# async def extract_data(response):
#     parsed_url = urlparse(response.url)
#     print("parsed_url -- ", parsed_url)
#     query_params = parse_qs(parsed_url.query)
#     print("query_params -- ", query_params)
#     # On vérifie si 'queryid=products' is in the URL
#     if "queryid" in query_params and query_params["queryid"][0] == "products":
#         data = await response.json()
#         print("JSON data found in : ", response.url)
#         print("data : ", data)
#     else:
#         print("queryid n'est présent !")

async def extract_data(response):
    print(response)
    parsed_url = urlparse(response.url)
    #print("parsed_url -- ", parsed_url)
    query_params = parse_qs(parsed_url.query)
    print("query_params -- ", query_params)
    #print("query_params 0 -- ", query_params["key"][0])

    data0 = []
    
    content_type = response.headers.get("content-type", "")
    print("content-type -- ", content_type)
    # # On vérifie si 'queryid=products' is in the URL
    #if "queueittoken" in query_params or "api" in response.url:
    #if "querytype=products" in response.url.lower():
    #if "queueittoken" in response.url.lower():
    #if "darty-prod" in response.url.lower():
    # if "datadome" in response.url.lower():
    #if "application/json" in content_type or "api" in response.url:
    if "application/json" in content_type or "api" in response.url:
        data = await response.json()
        data0 = data0.append(data)
        print("Les données JSON trouvé dans : ", response.url)
        print("data : ", data)
    else:
        print("Mot cherché non présent !")

    # #if data != {}:
    file_name = "extracted_darty_data.json"
    with open(file_name, 'w', encoding="utf-8") as file:
        json.dump(data0, file)
    #else:
        #print("Pas de données enregistrées !")
    
    # file_name = "extracted_data.json"
    # with open(file_name, 'w') as file:
    #     json.dump()

async def handler():
    await page.evaluate("window.removeObstructionsForTestIfNeeded()")
    
async def scrape_website_product(url):
    #ua = "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:143.0) Gecko/20100101 Firefox/143.0"
    user_agent = {"User_Agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:143.0) Gecko/20100101 Firefox/143.0"}
    ua = UserAgent().random # sans random, il retourn un objet pas un str
    print(type(ua), " -- ", ua)
    async with async_playwright() as pw : 
        # On lance un navigateur
        browser = await pw.chromium.launch(headless=False, args=["--disable-blink-features=Automation Controlled"])
        #browser = await pw.chromium.launch_persistent_context(user_data_dir="/tmp/playwright", headless=False)
        #browser = await pw.firefox.launch(headless=False, args=["--start-maximized"])
        #context = await browser.new_context(user_agent=ua)
        #context = await browser.new_context(ignore_https_errors=True)
        context = await browser.new_context()
        await context.set_extra_http_headers({"User_Agent":ua})
        # On ouvre une page sur cette navigateur
        page = await browser.new_page()
        #page = await context.new_page()


        #await page.add_locator_handler(page.locator("body"), handler, no_wait_after=True)
        #await page.get_by_role("button", name="Tout accepter").click()
        
        # La page écoute et interagit avec l'événement 'response'
        page.on(event="response", f=lambda response: extract_data(response))
        await asyncio.sleep(1)
        # On passe l'url sur sur la barre d'adresse de la page
        await page.goto(url, wait_until="networkidle", timeout=0)# wait_until="networkidle", "domcontentloaded"
        #page.set_default_navigation_timeout(0)
        await asyncio.sleep(1)

        # Faire des screeshots
        #await page.screenshot(path=url+".png", full_page=True)
        
        # On s'assure que la page est chargé
        #await page.evaluate('window.scrollTo(0, document.body.scrollHeight);')
        #await scroll_to_bottom(page, 3)
        #await page.click("text=Voir tout")

        # Pour temu : 
        # cliquer sur 'Tout accepter'
        try : 
            # button = await page.wait_for_selector('text=Tout accepter')
            # #await page.click('text=Tout accepter')
            # await button.dispatch_event('click')
            # 
            # print("Cookies acceptés")
            # await page.evaluate("""
            #                 let banners = document.querySelectorAll('[id*="cookie"], 
            #                 [class*="cookie"], [id*="consent"], [class*="consent"]');
            #                 banners.forEach(b => b.remove());
            #                 """)
            # print("✅ Bandeau supprimé par script")
            # await asyncio.sleep(1)
            # await page.mouse.wheel(0, 200) # scroll un peu
            # await page.wait_for_timeout(500) # pause
            # await page.click("text=Tout accepter", force=True) 
            
            # await context.add_cookies([{
            #     "Name":"privacy_setting_detail",  #cookie_consent",
            #     "Value":"%7B%22firstPAds%22%3A1%2C...11111111011111111111%22%7D",
            #     "Domain":"www.temu.com",
            #     "HostOnly":"true",
            #     "HttpOnly":"false",
            #     "Path":"/",
            #     "Secure":"true",
            #     "Taille":"411"
            # }])

            for frame in page.frames:
                try:
                    #btn = await frame.query_selector("text=Tout accepter")
                    btn = await frame.query_selector("text="+re.search(".*?", "accepter", re.IGNORECASE).group())
                    if btn:
                        await btn.click()
                        print("Cookies acceptés dans un ifrmame")
                        break
                except:
                    pass
        except:
            print("Bouton 'Tout accepter' introuvable")
        
        # attendre que la page de la promotion s'affiche
        # await page.wait_for_selector('.Quitter la promotion')
        # await page.click('.Quitter la promotion')

        
        content = await page.content()
        print(content)
        #html = response.text()
        #print(html[:200])
        
        
        await asyncio.sleep(300)
        await context.close()
        await browser.close()

In [6]:
# block page by resource type e.g. image, stylesheet
BLOCK_RESOURCE_TYPES = [
    'beacon',
    'csp_report',
    'font',
    'image',
    'imageset',
    'media',
    'object',
    'texttrack',
    # we can even block stylesheets and script through it's not recommended:
    # 'stylesheet',
    # 'script',
    # 'xhr'
]

# we can also blok popular 3rd party resource like tracking
BLOCK_RESOURCE_NAMES = [
    'adzerk',
    'analytics',
    'cdn.api.twitter',
    'doubleclick',
    'exelator',
    'facebook',
    'fontawesome',
    'google',
    'google-analytics',
    'googletagmanager',
]

async def intercept_route(route):
    """ intercept all request and abort blocked ones """ 
    if route.request.reource_type in BLOCK_RESOURCE_TYPES:
        print(f'blocking background resource {route.resource} blocked type "{route.request.reource_type}"') 
        return route.abort()
    if any(key in route.request.url for key in BLOCK_RESOURCE_NAMES):
        print(f"blocking backgroud resource {route.resource} blocked name {route.request.url}")
        return route.abort()

    return route.continue_()

# Scrolling et loading...
async def scroll_to_bottom_and_load(page, n, text_to_click):
    last_height = await page.evaluate('document.body.scrollHeight')
    i = 1

    while i <= n:
        print(f'Page {i}...')
        # du début jusquà une position donnée
        await page.evaluate('window.scroll(0, document.body.scrollHeight)')
        asyncio.sleep(1)
        
        # position actuelle
        new_height = await page.evaluate('document.body.scrollHeight')

        if new_height == last_height:
            try: 
                await page.wait_for_selector("text="+text_to_click)
                await page.click("text="+text_to_click)
            except TimeoutError:
                print('Limite atteinte.')
                break
        # On réinitialise last_height pour la page suivante
        last_height = new_height        
        i+=1

async def website_scraper(url): #, text_bandeau="", selector):
    async with async_playwright() as pw:
        # create browser instance
        browser = await pw.chromium.launch(
            # we can choose either headful (with GUI) or headless mode:
            headless=False,
            #devtools=True
            args=['--disable-blink-']
        )
        # create context
        # Using context we can define page properties like viewport dimensions
        context = await browser.new_context(
            # most commun desktop viewport is 1920x1080
            viewport={"width":1920, "height":1080}
        )
        # create page aka browser tab wich we'll be using to do everything
        page = await context.new_page()

        # enable for intercepting for this page, **/* enable for all request
        page.on("**/*", intercept_route)
        
        # go to url
        #await page.goto("https://twitch.tv/directory/game/Art")
        await page.goto(url, timeout=0)
        await asyncio.sleep(1)

        await page.wait_for_selector("text=J'ACCEPTE", timeout=0)
        await page.click("text=J'ACCEPTE")
        await asyncio.sleep(1)
    
        # get html
        #print(await page.content())
        # wait for first result appear
        page.wait_for_selector("body", timeout=0)
        asyncio.sleep(1)
        
        await scroll_to_bottom_and_load(page, 6, "Voir plus de produits")
        asyncio.sleep(1)
       
        products = page.locator(".product")
        count = await products.count()
        print("nombre de produits : ", count)

        products_list = []

        for i in range(count):
            family = await products.nth(i).locator(".product .column.center .family").inner_text()
            brand = await products.nth(i).locator(".product .column.center .brand").inner_text()
            reference = await products.nth(i).locator(".product .column.center .reference").inner_text()
            average = await products.nth(i).locator(".product .column.center .rating .average").inner_text()
            reviewNumber = await products.nth(i).locator(".product .column.center .rating .count .deco").inner_text()
            seller = await products.nth(i).locator(".product .column.right .seller").inner_text()
            label = await products.nth(i).locator(".product .column.right .seller .label").inner_text()
            price_ttc = await products.nth(i).locator(".product .column.right .price_product .price.price_ttc").inner_text()
            
            products_list.append({"typeTelephone":family,
                                 "marque":brand,
                                 "reference":reference,
                                 "note":average,
                                 "nombreAvis":reviewNumber,
                                 "vendeur":seller,
                                 "nomDuVendeur":label,
                                 "prix":price_ttc})
            print(f"family : {family}\n brand : {brand}\n reference : {reference}\n"+
                  f"average : {average}\n deco : {reviewNumber}\n seller : {seller}\n"+
                  f"label : {label}\n price_ttc : {price_ttc}")
            
            
        with open("darty_html.csv", 'w', newline='', encoding="utf-8") as csvfile:
            fieldnames = products_list[0].keys()#["marque", "année", "prix"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(products_list)

        # await asyncio.sleep(300)
        # await context.close()
        # await browser.close()

In [22]:
"""
    Instanciation des variables
"""
#  Paramètres
# lire des mots clés sur un lien

robots_keywords = ["product", "products", "item", "itm", "kw", "produit", "produits", "listing", "dp", "catalog", "catalogue"]
base_url = "https://www.fnac.com" # "https://www.ebay.com", "https://www.amazon.com"


# Liste de probables mots qui peuvent se trouver dans les titres des cgu
cgu_keywords_list1 = ["interdit", "robots", "bots", "scraper", "scraping", "aspirer", "logiciels", "automatisé", 
                         "aspiration", "extraction de données", "automated access", "crawling", 
                         "data extraction", "spider", "personnel", "commercial", "partie substantielle", "partie non susbtantielle",
                         "usage", "licence", "propriété", "utilisation", "API"]
cgu_keywords_list = ["interdit", "robots", "bots", "scraper", "scraping", "aspiration de données", 
                     "logiciels automatisés", "extraction de données", "automated access", 
                     "crawling", "data extraction", "spider", "usage personnel", 
                     "usage commercial", "non commercial", "partie substantielle", 
                     "partie non susbtantielle", "API"]

darty_url, darty_cgu_path = "https://cuisine.darty.com", "/cgu" #"https://cadeau.darty-services.com", "/CGU.pdf"
fnac_url, fnac_cgu_path = "https://www.fnac.com", "/Help/cgu-fnac"
amazon_url, amazon_cgu_path = "https://www.amazon.com", "/gp/help/customer/display.html?nodeId=GLSBYFE9MGKKQXXM"
ebay_url, ebay_cgu_path = "https://www.ebay.fr", "/help/policies/member-behaviour-policies/user-agreement?id=4259"
off_url, off_cgu_path = "https://fr.openfoodfacts.org", "/conditions-d-utilisation"
etsy_url, etsy_cgu_path = "https://www.etsy.com", "/fr/legal/terms-of-use"
temu_url, temu_cgu_path = "https://www.temu.com", "/fr/terms-of-use.html?refer_page_name=support-center&refer_page_id=10318_1758844934157_ynijdhm7h4&refer_page_sn=10318&_x_sessn_id=s1w3r2xioe"
bts_url = "https://books.toscrape.com/"

#off_produits_url = "https://fr.openfoodfacts.org/cgi/search.pl?search_terms=produi&search_simple=1&action=process"
off_produits_url = "https://fr.openfoodfacts.org/produit/"
sts_items_url = "https://www.scrapethissite.com/pages/simple/"
nike_products_url = "https://www.nike.com/cz/en/w/mens-shoes-nik1zy7ok" #"https://www.nike.com/cz/en/men" #"https://www.nike.com/cz/en/t/air-max-plus-shoes-EWa8uypC/" #"https://www.nike.com/cz/en/w/mens-air-max-lifestyle-shoes-13jrmza6d8hznik1zy7ok"
darty1_products_url = "https://darty.com/nav/achat/informatique/index.html"
darty_products_url = "https://darty.com/nav/extra/list?flag=ODR&cat=89054&prix_barre=dcom_BonPlan-dcom_BONPLAN-dcom_OFDARTY&fa=294580-693-138552"
temu_products_url = "https://www.temu.com/fr/cell-phones-o3-2641.html?opt_level=2&title=T%C3%A9l%C3%A9phones%20portables&_x_enter_scene_type=cate_tab&leaf_type=bro&show_search_type=3&_x_sessn_id=hee7ujhfgf&refer_page_name=category&refer_page_id=10012_1759272164200_73ttyjc3sc&refer_page_sn=10012"


# base_url+"robots.txt"

# darty -> ok, usage de partie non substantiel
# temu -> ok, usage personnel et non commercial

In [23]:
"""
    Appel des fonctions
"""
if __name__=='__main__':
    # Main
    #robots_results(base_url, robots_keywords);
    #robots_keywords = robots_file_analysis(base_url, robots_keywords) 
    # pour éviter l'affichage du retur, on peut la fonction d'appel dans une variable ou mettre un ; à la fin
    #display_robots_results(base_url, robots_keywords)

    #cgu_analysis(url_fnac, fnac_cgu_path)
    #await cgu_analysis_auto(url_temu, temu_cgu_path, cgu_keywords_list) # url_fnac, fnac_cgu_path
    
    #await display_cgu_results(url_off, off_cgu_path, cgu_keywords_list)

    #await scrape_website_product(temu_products_url) #"https://www.temu.com/") #darty_products_url) # temu_products_url

    await website_scraper(darty_products_url)
    
    # "usage personnel ou non commercial", "logiciels manuels ou automatisés"
    # soup = BeautifulSoup(page, "html.parser")

    """
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) 
Chrome/135.0.0.0 Safari/537.36"
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) 
Chrome/135.0.0.0 Safari/537.36"
        "Mozilla/5.0 (iPhone; CPU iPhone OS 16_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) 
Version/16.4 Mobile/15E148 Safari/604.1"
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) 
Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0"
        "Mozilla/5.0 (iPhone; CPU iPhone OS 18_3_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like 
Gecko) Version/18.3.1 Mobile/15E148 Safari/604.1"
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 
Safari/537.36"
    
        query_params -- 
{
    'flag': ['ODR'],
    'cat': ['89054'],
    'prix_barre': ['dcom_BonPlan-dcom_BONPLAN-dcom_OFDARTY'],
    'fa': ['294580-693-138552']
        }
    """

# class="text ellipsis ellipsis-2lines
# <span class="trigger-text ellipsis ellipsis-2lines">Caractéristiques</span>
# <span class="average">4,7/5</span>
# <span class="deco">16 avis</span>
# <span class="family">Smartphone</span>
# <span class="brand">Samsung</span>



TargetClosedError: Page.wait_for_selector: Target page, context or browser has been closed
Call log:
  - waiting for locator("text=J'ACCEPTE") to be visible


In [15]:
# Erreurs

# TypeError: object of type 'Page' has no len()
# /tmp/ipykernel_121818/2292587623.py:29: RuntimeWarning: coroutine 'Locator.all' was never awaited
#   for row in page.get_by_role("listitem").all():
# RuntimeWarning: Enable tracemalloc to get the object allocation traceback

# TimeoutError: Page.goto: Timeout 30000ms exceeded.
# Call log:
#   - navigating to "https://www.ebay.fr/help/policies/member-behaviour-policies/user-agreement?id=4259", waiting until "networkidle"

In [17]:
# l'url du site sur lequel 
url = ""

# fonction qui l'url et renvoi un dataFrame
def scraper_site(url) : 
    url = url
    
    # Créer un objet browser, pour se placer sur le navigateur
    browser = mechanicalsoup.Browser()
    # Ouverture de la page web de l'url sur le navigateur
    page = browser.get(url)
    # Récupérer la page html avec l'objet soup de Beautiful
    html = page.soup
    
    

In [18]:
#help(requests)
BeautifulSoup?

Init signature:
BeautifulSoup(
    markup='',
    features=None,
    builder=None,
    parse_only=None,
    from_encoding=None,
    exclude_encodings=None,
    element_classes=None,
    **kwargs,
)
Docstring:     
A data structure representing a parsed HTML or XML document.

Most of the methods you'll call on a BeautifulSoup object are inherited from
PageElement or Tag.

Internally, this class defines the basic interface called by the
tree builders when converting an HTML/XML document into a data
structure. The interface abstracts away the differences between
parsers. To write a new tree builder, you'll need to understand
these methods as a whole.

These methods will be called by the BeautifulSoup constructor:
  * reset()
  * feed(markup)

The tree builder may call these methods from its feed() implementation:
  * handle_starttag(name, attrs) # See note about return value
  * handle_endtag(name)
  * handle_data(data) # Appends to the current data node
  * endData(containerClass) # Ends